In [1]:
# mnist_cnn
# MNIST and Convolutional Neural Network

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(5)


In [2]:
# mnist 데이터 가져오기
mnist = tf.keras.datasets.mnist
(x_train,y_train), (x_test,y_test) = mnist.load_data()
print(x_train.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)


In [3]:
# one-hot 인코딩
nb_classes = 10     # 분류 class의 갯수(0 ~ 9)
Y_one_hot = tf.one_hot(y_train,nb_classes) # (60000, 10)
print(Y_one_hot.shape)                     # (60000, 10) , 2차원


# X값의 타입을 float형으로 변환
x_train = tf.cast(x_train,dtype=tf.float32)
print(x_train.shape,x_train.dtype)  # 'float32'

x_test = tf.cast(x_test,dtype=tf.float32)
print(x_test.shape,x_test.dtype)  # 'float32


(60000, 10)
(60000, 28, 28) <dtype: 'float32'>
(10000, 28, 28) <dtype: 'float32'>


In [4]:
# X값의 shape을 4차원으로 변환
X_img = tf.reshape(x_train,[-1,28,28,1])
print(X_img.shape)

(60000, 28, 28, 1)


In [5]:
# Layer 1  : conv2d - relu - max_pool
# (?,28,28,1) ---> (?,14,14,32)

# <1> conv2d
# L1 image shape : (?,28,28,1)
# filter : (3,3,1,32) , 필터 32개
# strides : (1,1,1,1), padding ='SAME'
# 출력 이미지 : (28+2 - 3)/1 + 1 = 28,   --> (?,28,28,32)
W1 = tf.Variable(tf.random.normal([3,3,1,32]),name='weight1')  # filter
def L1_conv2d(X):
    return tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')

# <2> relu
def L1_relu(X):
    return tf.nn.relu(L1_conv2d(X))   # shape 변화가 없다

# <3> max_pool
# input image : (?,28,28,32)
# kernel size : (1,2,2,1), strides : (1,2,2,1), padding ='SAME'
# 출력 이미지 : (28+1 - 2)/2 + 1 = 14  ---> (?,14,14,32)
def L1_MaxPool(X):
    return tf.nn.max_pool(L1_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')    

In [6]:
# Layer2  : conv2d - relu - max_pool
# (?,14,14,32) --> (?,14,14,64) --> (?,7,7,64) --> (?,7*7*64) 

# <1> conv2d
# L2 image shape : (?,14,14,32)
# filter : (3,3,32,64) , 필터 64개
# strides : (1,1,1,1), padding ='SAME'
# 출력 이미지 : (14+2 - 3)/1 + 1 = 14,   --> (?,14,14,64)
W2 = tf.Variable(tf.random.normal([3,3,32,64]),name='weight2')  # filter
def L2_conv2d(X):
    return tf.nn.conv2d(L1_MaxPool(X),W2,strides=[1,1,1,1],padding='SAME')

# <2> relu
def L2_relu(X):
    return tf.nn.relu(L2_conv2d(X))   # shape 변화가 없다

# <3> max_pool
# input image : (?,14,14,64)
# kernel size : (1,2,2,1), strides : (1,2,2,1), padding ='SAME'
# 출력 이미지 : (14+1 - 2)/2 + 1 = 7  ---> (?,7,7,64)
def L2_MaxPool(X):
    return tf.nn.max_pool(L2_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') 

# <4> flatten layer  : 다차원 배열을 2차원으로 변환하여 FC layer에 전달한다
def L2_flat(X):
    return tf.reshape(L2_MaxPool(X),[-1,7*7*64])

In [7]:
# Layer 3 : FC(Fully Connected Layer)
# (?,7*7*64) * (7*7*64,10 ) --> (?,10)
nb_classes = 10
W3 = tf.Variable(tf.random.normal([7*7*64,nb_classes]), name ='weight3')
b = tf.Variable(tf.random.normal([nb_classes]), name = 'bias')

In [8]:
# hypothesis 예측 함수 : H(X) = softmax(W*X + b)
def logits(X):
    return tf.matmul(L2_flat(X),W3) + b

def hypothesis(X):
    return   tf.nn.softmax(logits(X))  

In [9]:
# 방법 2. batch 사이즈로 나누어 학습, 효율적 이며 학습 시간 단축
# 학습 시작

training_epochs = 50
batch_size = 600

# 경사 하강법
# learning_rate(학습율) 을 0.01로 설정하여 optimizer 객체를 생성
optimizer = tf.keras.optimizers.Adam(lr=0.01)

Y_one_hot = tf.one_hot(y_train,nb_classes)  # (60000,10)

print('***** Start Learning!!')
for epoch in range(training_epochs):  # 50
    
    avg_cost = 0
    
    # 100 = 60000/600
    total_batch = int(x_train.shape[0]/batch_size)
    for k in range(total_batch):  # 100 
        batch_xs = x_train[0+k*batch_size:batch_size + k*batch_size]
        batch_ys = Y_one_hot[0+k*batch_size:batch_size + k*batch_size]
        
        # X값의 shape을 4차원으로 변환 코드 추가
        X_img = tf.reshape(batch_xs,[-1,28,28,1])
        
        # 비용함수
        def cost_func_batch():
            cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits(X_img),
                                                     labels=batch_ys) 
            cost = tf.reduce_mean(cost_i)
            return cost    
        
        # cost를 minimize한다
        optimizer.minimize(cost_func_batch,var_list=[W1,W2,W3,b])
        avg_cost += cost_func_batch().numpy()/total_batch
    
    print('Epoch:','%04d'%(epoch + 1),'cost:','{:.9f}'.format(avg_cost))
          
print('***** Learning Finished')

***** Start Learning!!
Epoch: 0001 cost: 16359.360611572
Epoch: 0002 cost: 2609.244541016
Epoch: 0003 cost: 1503.309845581
Epoch: 0004 cost: 1013.575948486
Epoch: 0005 cost: 746.071317902
Epoch: 0006 cost: 577.549841995
Epoch: 0007 cost: 453.436385994
Epoch: 0008 cost: 364.561368103
Epoch: 0009 cost: 308.943035088
Epoch: 0010 cost: 253.859949150
Epoch: 0011 cost: 215.736172752
Epoch: 0012 cost: 189.695848408
Epoch: 0013 cost: 151.001017895
Epoch: 0014 cost: 123.021624918
Epoch: 0015 cost: 95.155155702
Epoch: 0016 cost: 85.175316920
Epoch: 0017 cost: 63.144783616
Epoch: 0018 cost: 64.453414921
Epoch: 0019 cost: 51.621211880
Epoch: 0020 cost: 52.028042244
Epoch: 0021 cost: 50.492375683
Epoch: 0022 cost: 47.242263544
Epoch: 0023 cost: 37.381401564
Epoch: 0024 cost: 35.557780277
Epoch: 0025 cost: 23.786104871
Epoch: 0026 cost: 17.212622674
Epoch: 0027 cost: 13.829266647
Epoch: 0028 cost: 9.983195732
Epoch: 0029 cost: 10.512800937
Epoch: 0030 cost: 8.796221958
Epoch: 0031 cost: 6.889642160


In [10]:
# 예측  , accuracy computation

def predict(X):
    return tf.argmax(hypothesis(X),1) 

Y_one_hot = tf.one_hot(y_test,nb_classes)  # (10000, 10)
# print(Y_one_hot.shape)

# X값의 shape을 4차원으로 변환 코드 추가
X_img = tf.reshape(x_test,[-1,28,28,1])

correct_predict = tf.equal(predict(X_img),tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32))

print('***** Predict')

pred = predict(X_img).numpy()
print(pred,y_test)
print("Accuracy:",accuracy.numpy()) # Accuracy: 0.981

***** Predict
[7 2 1 ... 4 5 6] [7 2 1 ... 4 5 6]
Accuracy: 0.981


In [11]:
# 정확도 비교

# [1] softmax 사용
# 1 layers              -------> Accuracy  : 0.8871
# 4 layers  sigmoid     -------> Accuracy  : 0.9033
# 4 layers  relu        -------> Accuracy  : 0.9534  

# [2] CNN 사용
# 3 layers              -------> Accuracy  : 0.981
# 5 layers              -------> Accuracy  : 

# [3] Keras에서 CNN 사용
#
# VGGNet